# Code to generate figure 3 for the paper

In [1]:
# setup all the imports
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import yaml
import urllib.request
import matplotlib.font_manager
flist = matplotlib.font_manager.get_font_names()
from tempfile import NamedTemporaryFile
import urllib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LongitudeFormatter, LatitudeFormatter
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import matplotlib.patches as patches
from extremeweatherbench import evaluate, utils, cases, defaults
sns.set_theme(style='whitegrid')
from shapely.geometry import Polygon
import shapely
from pathlib import Path

# make the basepath - change this to your local path
basepath = Path.home() / 'ExtremeWeatherBench' / ''
basepath = str(basepath) + '/'

In [2]:
# %load /home/amy/ExtremeWeatherBench/docs/notebooks/figure_targets.py
import logging

import numpy as np
import xarray as xr

from extremeweatherbench import inputs, metrics

# Suppress noisy log messages
logging.getLogger("urllib3.connectionpool").setLevel(logging.CRITICAL)
logging.getLogger("botocore.httpchecksum").setLevel(logging.CRITICAL)


def _preprocess_bb_cira_forecast_dataset(ds: xr.Dataset) -> xr.Dataset:
    """An example preprocess function that renames the time coordinate to lead_time,
    creates a valid_time coordinate, and sets the lead time range and resolution not
    present in the original dataset.

    Args:
        ds: The forecast dataset to rename.

    Returns:
        The renamed forecast dataset.
    """
    ds = ds.rename({"time": "lead_time"})
    # The evaluation configuration is used to set the lead time range and resolution.
    ds["lead_time"] = np.array(
        [i for i in range(0, 241, 6)], dtype="timedelta64[h]"
    ).astype("timedelta64[ns]")
    return ds


# GHCN targets
ghcn_heatwave_target = inputs.GHCN(
    source=inputs.DEFAULT_GHCN_URI,
    variables=["surface_air_temperature"],
    variable_mapping={"t2": "surface_air_temperature"},
    storage_options={},
)

ghcn_freeze_target = inputs.GHCN(
    source=inputs.DEFAULT_GHCN_URI,
    variables=[
        "surface_air_temperature",
        "surface_eastward_wind",
        "surface_northward_wind",
    ],
    variable_mapping={
        "surface_temperature": "surface_air_temperature",
        "surface_eastward_wind": "surface_eastward_wind",
        "surface_northward_wind": "surface_northward_wind",
    },
    storage_options={},
)

# LSR and PPH targets
lsr_target = inputs.LSR(
    source=inputs.LSR_URI,
    variables=["local_storm_reports"],
    variable_mapping={},
    storage_options={"remote_options": {"anon": True}},
)

pph_target = inputs.PPH(
    source=inputs.PPH_URI,
    variables=["practically_perfect_hindcast"],
    variable_mapping={},
    storage_options={"remote_options": {"anon": True}},
)

# IBTrACS target
ibtracs_target = inputs.IBTrACS(
    source=inputs.IBTRACS_URI,
    variables=[
        "surface_wind_speed",
        "air_pressure_at_mean_sea_level",
    ],
    variable_mapping=inputs.IBTrACS_metadata_variable_mapping,
    storage_options={},
)

# Forecast Examples

cira_heatwave_forecast = inputs.KerchunkForecast(
    source="gs://extremeweatherbench/FOUR_v200_GFS.parq",
    variables=["surface_air_temperature"],
    variable_mapping={"t2": "surface_air_temperature"},
    storage_options={"remote_protocol": "s3", "remote_options": {"anon": True}},
    preprocess=_preprocess_bb_cira_forecast_dataset,
)

cira_freeze_forecast = inputs.KerchunkForecast(
    source="gs://extremeweatherbench/FOUR_v200_GFS.parq",
    variables=[
        "surface_air_temperature",
        "surface_eastward_wind",
        "surface_northward_wind",
    ],
    variable_mapping={
        "t2": "surface_air_temperature",
        "10u": "surface_eastward_wind",
        "10v": "surface_northward_wind",
    },
    storage_options={"remote_protocol": "s3", "remote_options": {"anon": True}},
    preprocess=_preprocess_bb_cira_forecast_dataset,
)

# Severe convection forecast
cira_severe_convection_forecast = inputs.KerchunkForecast(
    source="gs://extremeweatherbench/FOUR_v200_GFS.parq",
    variables=[],
    variable_mapping={
        "t": "air_temperature",
        "t2": "surface_air_temperature",
        "z": "geopotential",
        "r": "relative_humidity",
        "u": "eastward_wind",
        "v": "northward_wind",
        "10u": "surface_eastward_wind",
        "10v": "surface_northward_wind",
    },
    storage_options={"remote_protocol": "s3", "remote_options": {"anon": True}},
)

cira_tropical_cyclone_forecast = inputs.KerchunkForecast(
    source="gs://extremeweatherbench/FOUR_v200_GFS.parq",
    variables=[
        "surface_wind_speed",
        "air_pressure_at_mean_sea_level",
    ],
    variable_mapping={
        "t": "air_temperature",
        "t2": "surface_air_temperature",
        "z": "geopotential",
        "r": "relative_humidity",
        "u": "eastward_wind",
        "v": "northward_wind",
        "10u": "surface_eastward_wind",
        "10v": "surface_northward_wind",
    },
    storage_options={"remote_protocol": "s3", "remote_options": {"anon": True}},
)

FIGURE_TARGETS_EVALUATION_OBJECTS = [
    inputs.EvaluationObject(
        event_type="heat_wave",
        metric_list=[
            metrics.MaximumMAE,
            metrics.RMSE,
            metrics.OnsetME,
            metrics.DurationME,
            metrics.MaxMinMAE,
        ],
        target=ghcn_heatwave_target,
        forecast=cira_heatwave_forecast,
    ),
    inputs.EvaluationObject(
        event_type="freeze",
        metric_list=[
            metrics.MinimumMAE,
            metrics.RMSE,
            metrics.OnsetME,
            metrics.DurationME,
        ],
        target=ghcn_freeze_target,
        forecast=cira_freeze_forecast,
    ),
    inputs.EvaluationObject(
        event_type="severe_convection",
        metric_list=[
            metrics.CSI,
            metrics.FAR,
            metrics.RegionalHitsMisses,
            metrics.HitsMisses,
        ],
        target=lsr_target,
        forecast=cira_severe_convection_forecast,
    ),
    inputs.EvaluationObject(
        event_type="severe_convection",
        metric_list=[
            metrics.CSI,
            metrics.FAR,
            metrics.RegionalHitsMisses,
            metrics.HitsMisses,
        ],
        target=pph_target,
        forecast=cira_severe_convection_forecast,
    ),
    inputs.EvaluationObject(
        event_type="tropical_cyclone",
        metric_list=[
            metrics.EarlySignal,
            metrics.LandfallDisplacement,
            metrics.LandfallTimeME,
            metrics.LandfallIntensityMAE,
        ],
        target=ibtracs_target,
        forecast=cira_tropical_cyclone_forecast,
    ),
]


In [7]:
case_dict['cases'][0]

{'case_id_number': 1,
 'title': '2021 Pacific Northwest',
 'start_date': datetime.datetime(2021, 6, 20, 0, 0),
 'end_date': datetime.datetime(2021, 7, 3, 0, 0),
 'location': {'type': 'centered_region',
  'parameters': {'latitude': 47.6062,
   'longitude': 237.6679,
   'bounding_box_degrees': 5}},
 'event_type': 'heat_wave'}

In [13]:
# load in all of the events in the yaml file
case_dict = utils.load_events_yaml()
heat_test = {"cases": case_dict["cases"][:1]}

ewb = evaluate.ExtremeWeatherBench(heat_test, FIGURE_TARGETS_EVALUATION_OBJECTS)

This function is deprecated and will be removed in a future release. Please use cases.load_ewb_events_yaml_into_case_collection instead.
This function is deprecated and will be removed in a future release. Please use cases.read_incoming_yaml instead.


In [14]:
heat_test

{'cases': [{'case_id_number': 1,
   'title': '2021 Pacific Northwest',
   'start_date': datetime.datetime(2021, 6, 20, 0, 0),
   'end_date': datetime.datetime(2021, 7, 3, 0, 0),
   'location': {'type': 'centered_region',
    'parameters': {'latitude': 47.6062,
     'longitude': 237.6679,
     'bounding_box_degrees': 5}},
   'event_type': 'heat_wave'}]}

In [15]:
ewb.run()

  0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 